# DAT_SF_13 - Lab 12

### Recommender Systems


1. Simple similarity based recommendation system
2. Recsys


## Similarity based Recommendation System: Beers


Let's build a recommendation system to recommend types of beers based on user reviews

Usual imports (numpy, pandas)

In [19]:
import pandas as pd
import numpy as np

First of all let's get the data

In [20]:
! curl -O https://s3.amazonaws.com/demo-datasets/beer_reviews.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 27.3M  100 27.3M    0     0  2714k      0  0:00:10  0:00:10 --:--:-- 3413k


Import data in a pandas dataframe called "allbeers". Use the compression keyword

In [21]:
allbeers = pd.read_csv('beer_reviews.tar.gz', compression = 'gzip')

Let's look at the data

In [22]:
allbeers.head()

,beer_reviews/,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,10325,Vecchio Birraio,1234817823,1.5,2.0,2.5,stcules,Hefeweizen,1.5,1.5,Sausa Weizen,5.0,47986
1,10325,Vecchio Birraio,1235915097,3.0,2.5,3.0,stcules,English Strong Ale,3.0,3.0,Red Moon,6.2,48213
2,10325,Vecchio Birraio,1235916604,3.0,2.5,3.0,stcules,Foreign / Export Stout,3.0,3.0,Black Horse Black Beer,6.5,48215
3,10325,Vecchio Birraio,1234725145,3.0,3.0,3.5,stcules,German Pilsener,2.5,3.0,Sausa Pils,5.0,47969
4,1075,Caldera Brewing Company,1293735206,4.0,4.5,4.0,johnmichaelsen,American Double / Imperial IPA,4.0,4.5,Cauldron DIPA,7.7,64883


Let's restrict this to the top 250 beers. Use the value_counts() method select the top 250 beers.
Assign the selected beers to a dataset called df

In [24]:
n = 250
top_n = allbeers['beer_name'].value_counts().index[:n]

top_n

Index([u'90 Minute IPA', u'India Pale Ale', u'Old Rasputin Russian Imperial Stout', u'Sierra Nevada Celebration Ale', u'Two Hearted Ale', u'Stone Ruination IPA', u'Arrogant Bastard Ale', u'Sierra Nevada Pale Ale', u'Stone IPA (India Pale Ale)', u'Pliny The Elder', u'Founders Breakfast Stout', u'Pale Ale', u'Sierra Nevada Bigfoot Barleywine Style Ale', u'La Fin Du Monde', u'60 Minute IPA', u'Storm King Stout', u'Duvel', u'Brooklyn Black Chocolate Stout', u'Bell's Hopslam Ale', u'Samuel Adams Boston Lager', u'Stone Imperial Russian Stout', u'HopDevil Ale', u'Chocolate Stout', u'Imperial Stout', u'Young's Double Chocolate Stout', u'Dead Guy Ale', u'St. Bernardus Abt 12', u'Guinness Draught', u'Trappistes Rochefort 10', u'Ayinger Celebrator Doppelbock', u'Sierra Nevada Torpedo Extra IPA', u'Samuel Smith's Oatmeal Stout', u'Chimay Grande Réserve (Blue)', u'Weihenstephaner Hefeweissbier', u'Racer 5 India Pale Ale', u'Tröegs Nugget Nectar', u'Founders KBS (Kentucky Breakfast Stout)', u'Orval 

How big is this dataset?

In [30]:
df = allbeers[allbeers['beer_name'].isin(top_n)]

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 355275 entries, 798 to 1586564
Data columns (total 13 columns):
beer_reviews/         355275 non-null object
brewery_name          355275 non-null object
review_time           355275 non-null float64
review_overall        355275 non-null float64
review_aroma          355275 non-null float64
review_appearance     355275 non-null float64
review_profilename    355175 non-null object
beer_style            355275 non-null object
review_palate         355275 non-null float64
review_taste          355275 non-null float64
beer_name             355275 non-null object
beer_abv              353477 non-null float64
beer_beerid           355275 non-null float64
dtypes: float64(8), object(5)

In [33]:
allbeers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1586630 entries, 0 to 1586629
Data columns (total 13 columns):
beer_reviews/         1586629 non-null object
brewery_name          1586611 non-null object
review_time           1586614 non-null float64
review_overall        1586614 non-null float64
review_aroma          1586614 non-null float64
review_appearance     1586614 non-null float64
review_profilename    1586266 non-null object
beer_style            1586614 non-null object
review_palate         1586614 non-null float64
review_taste          1586614 non-null float64
beer_name             1586614 non-null object
beer_abv              1518829 non-null float64
beer_beerid           1586614 non-null float64
dtypes: float64(8), object(5)

### Pivot Table

Aggregate the data in a pivot table using the pivot_table method. Display the mean review_overall for each beer_name aggregating the review_overall values by review_profilename. Use the mean as aggregator.

In [43]:
df_wide = pd.pivot_table(df, values=["review_overall"],
                         columns = ["beer_name", "review_profilename"], 
                         aggfunc=np.mean).unstack()
                        

In [42]:
df.wide.shape

AttributeError: 'DataFrame' object has no attribute 'wide'

Display the head of the pivot table, but only for 5 users (columns are users)

In [ ]:
df_wide.ix[0:5, 0:5]

### Discussion: what do you notice in this table?

#### Data munging
Set Nans to zero

In [ ]:
df_wide = df_wide 

Check that columns are users

Check that rows are beers

### Calculate distance between beers

We're going to use cosine_similarity from scikit-learn to compute the distance between all beers

Imports

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import manhattan_distances
from sklearn.metrics.pairwise import euclidean_distances

Apply cosine similarity to df_wide to calculate pairwise distances

In [ ]:
dist = cosine_similarity(df_wide)
dists

### Discussion: what type of object is dists?

Convert dists to a Pandas DataFrame, use the index as column index as well (distances are a square matrix).

In [ ]:
dists = pd.DataFrame(dists, columns = df_wide.index)

dist.index = dists.columns

dist

In [ ]:
dist

Select some beers and look their distances to other beers

Sum the distances of my favourite beers by row, to have one distance from each beer in the sample

#### Performance

Optional: which one is faster? use ```%timeit``` to check

#### Ranking

Sort summed beers from best to worse

Filter out the beers used as input and transform to list

Define a function that does what we just did for an arbitrary input list of beers. it should also receive the maximum number of beers requested n as optional parameter.

Test your function. Find the 10 beers most similar to "120 Minute IPA"

Cool, let's try again with the 10 beers most similar to ["Coors Light", "Bud Light", "Amstel Light"]

Optional: register an account on yhat and deploy your model following the instructions [here](https://docs.yhathq.com/python/examples/beer-recommender) and [here](http://nbviewer.ipython.org/gist/glamp/20a18d52c539b87de2af)

## Recsys

A python library for implementing a recommender system

In [ ]:
"""
##install python-recsys

### first install dependencies

pip install csc-pysparse networkx divisi2

### then install recsys
git clone https://github.com/python-recsys/python-recsys.git
cd python-recsys/

python setup.py install
"""

Load recsys.algotihm, set VERBOSE = True import SVD class

In [ ]:
import recsys.algorithm
recsys.algorithm.VERBOSE = True
from recsys.algorithm.factorize import SVD

Let's look at the files

In [ ]:
! ls movielens

Import 'movies.dat' to a 'movies' pandas dataframe. Make sure you name the columns, use the correct separator and define the index.

In [ ]:
movies.head()

Import 'ratings.dat' to a 'ratings' pandas dataframe. Make sure you name the columns, use the correct separator.

In [ ]:
ratings.head()

Initialize an SVD instance

In [ ]:
svd = SVD()

Populate it with the data from the ratings dataset, using the built in load_data method

Compute SVD

$M=U \Sigma V^T$:

In [ ]:
k = 100


you can also save the output SVD model (in a zip file)

In [ ]:
# svd.compute(k=k, min_values=10, pre_normalize=None, mean_center=True, post_normalize=True, savefile='/tmp/movielens')

Reload a saved model:

In [ ]:
# svd2 = SVD(filename='/tmp/movielens')

Find the ITEMID number for "Toy Story (1995)"

Find the ITEMID number for "Bug's Life, A (1998)"

Compute similarity between the two movies

Get movies similar to Toy Story

Predict rating for a given user and movie, $\hat{r}_{ui}$

Recommend non rated movies to a user

Which users should see Toy Story? (e.g. which users -that have not rated Toy Story- would give it a high rating?)

Find out more here: [https://github.com/ocelma/python-recsys](https://github.com/ocelma/python-recsys)